In [1]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


## Make JSON function call

### default example

In [3]:
from google.colab import userdata
from groq import Groq
import os
import json

client = Groq(api_key = userdata.get('GROQ_API_KEY'))
MODEL = 'llama3-70b-8192'


# Example dummy function hard coded to return the score of an NBA game
def get_game_score(team_name):
    """Get the current score for a given NBA game"""
    if "warriors" in team_name.lower():
        return json.dumps({"game_id": "401585601", "status": 'Final', "home_team": "Los Angeles Lakers", "home_team_score": 121, "away_team": "Golden State Warriors", "away_team_score": 128})
    elif "lakers" in team_name.lower():
        return json.dumps({"game_id": "401585601", "status": 'Final', "home_team": "Los Angeles Lakers", "home_team_score": 121, "away_team": "Golden State Warriors", "away_team_score": 128})
    elif "nuggets" in team_name.lower():
        return json.dumps({"game_id": "401585577", "status": 'Final', "home_team": "Miami Heat", "home_team_score": 88, "away_team": "Denver Nuggets", "away_team_score": 100})
    elif "heat" in team_name.lower():
        return json.dumps({"game_id": "401585577", "status": 'Final', "home_team": "Miami Heat", "home_team_score": 88, "away_team": "Denver Nuggets", "away_team_score": 100})
    else:
        return json.dumps({"team_name": team_name, "score": "unknown"})

def run_conversation(user_prompt):
    # Step 1: send the conversation and available functions to the model
    messages=[
        {
            "role": "system",
            "content": "You are a function calling LLM that uses the data extracted from the get_game_score function to answer questions around NBA game scores. Include the team and their opponent in your response."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_game_score",
                "description": "Get the score for a given NBA game",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "team_name": {
                            "type": "string",
                            "description": "The name of the NBA team (e.g. 'Golden State Warriors')",
                        }
                    },
                    "required": ["team_name"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_game_score": get_game_score,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                team_name=function_args.get("team_name")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )  # get a new response from the model where it can see the function response
        return second_response.choices[0].message.content

user_prompt = "What was the score of the Warriors game?"
print(run_conversation(user_prompt))



The score of the Warriors game was 128-121, where the Golden State Warriors defeated the Los Angeles Lakers.


### Timeline function  

We use Pydantic to define the model, and check the LLM json for validity.  

In [61]:
from google.colab import userdata
from groq import Groq
from pydantic import BaseModel, ValidationError, Field
from typing import List
import os
import json

client = Groq(api_key=userdata.get('GROQ_API_KEY'))
MODEL = 'llama3-70b-8192'

class TimelineEvent(BaseModel):
    timestamp: str = Field(..., description="Date of the event")
    title: str = Field(..., description="Title of the event")
    description: str = Field(..., description="Short description of the event")

class Timeline(BaseModel):
    timeline: List[TimelineEvent]

# Example dummy function hard coded to return the score of an NBA game
def save_timeline(timeline_data):
    try:
        timeline = Timeline(**timeline_data)
        return timeline
    except ValidationError as e:
        return {"error": e.errors()}

def run_conversation(user_prompt):
    # Step 1: send the conversation and available functions to the model
    messages = [
        {
            "role": "system",
            "content": "You are a function calling LLM that saves timelines using the function save_timeline. When prompted to give the timeline for an event, save it using the save_timeline function."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "name": "save_timeline",
            "description": "Save a timeline to database.",
            "parameters": {
                "type": "object",
                "properties": {
                    "timeline": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "timestamp": {
                                    "type": "string",
                                    "description": "date of the event"
                                },
                                "title": {
                                    "type": "string",
                                    "description": "title of the event"
                                },
                                "description": {
                                    "type": "string",
                                    "description": "short description of the event"
                                }
                            },
                            "required": ["timestamp", "title", "description"]
                        }
                    }
                },
                "required": ["timeline"]
            }
        }
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        functions=tools,
        max_tokens=4096
    )

    try:
      response_message = response.choices[0].message
      timeline_data = json.loads(response_message.function_call.arguments)
    except Exception as e:
      print(e)
      return response

    try:
      result = save_timeline(timeline_data)
      return result
    except Exception as e:
      print(e)
      return response

    return response

# Example usage
user_prompt = "Provide a timeline for the events of September 11, 2001."
result = run_conversation(user_prompt)
print(json.dumps(json.loads(result.json()), indent=2))

{
  "timeline": [
    {
      "timestamp": "2001-09-11 07:59:00",
      "title": "Flight 11 Departs",
      "description": "American Airlines Flight 11 takes off from Boston's Logan International Airport"
    },
    {
      "timestamp": "2001-09-11 08:14:00",
      "title": "Flight 175 Departs",
      "description": "United Airlines Flight 175 takes off from Boston's Logan International Airport"
    },
    {
      "timestamp": "2001-09-11 08:20:00",
      "title": "Flight 77 Departs",
      "description": "American Airlines Flight 77 takes off from Washington Dulles International Airport"
    },
    {
      "timestamp": "2001-09-11 08:42:00",
      "title": "Flight 93 Departs",
      "description": "United Airlines Flight 93 takes off from Newark International Airport"
    },
    {
      "timestamp": "2001-09-11 08:46:00",
      "title": "Flight 11 Crashes into North Tower",
      "description": "American Airlines Flight 11 crashes into the North Tower of the World Trade Center"
    },

## Make two successful test and one error test  

We want to see if this functions works for different applications, and what happens when the LLM doesn't serve a Timeline object.

In [62]:
user_prompt = "Provide a timeline for the flie of Napoleon Bonaparte"
result = run_conversation(user_prompt)
print(json.dumps(json.loads(result.json()), indent=2))

{
  "timeline": [
    {
      "timestamp": "1769-08-15",
      "title": "Birth",
      "description": "Napoleon was born"
    },
    {
      "timestamp": "1785-10-28",
      "title": "Education",
      "description": "Napoleon graduated from \u00c9cole Militaire"
    },
    {
      "timestamp": "1793-07-07",
      "title": "Rise to Power",
      "description": "Napoleon became a general"
    },
    {
      "timestamp": "1799-11-09",
      "title": "Coup d'\u00e9tat",
      "description": "Napoleon overthrew the French Directory"
    },
    {
      "timestamp": "1804-12-02",
      "title": "Coronation",
      "description": "Napoleon was crowned Emperor of France"
    },
    {
      "timestamp": "1813-10-16",
      "title": "Defeat",
      "description": "Napoleon defeated at the Battle of Leipzig"
    },
    {
      "timestamp": "1814-04-06",
      "title": "Exile",
      "description": "Napoleon exiled to Elba"
    },
    {
      "timestamp": "1815-02-26",
      "title": "Escape",
   

In [63]:
user_prompt = "Provide a timeline for an apple pie recipe"
result = run_conversation(user_prompt)
print(json.dumps(json.loads(result.json()), indent=2))

{
  "timeline": [
    {
      "timestamp": "12:00 pm",
      "title": "Preheat oven",
      "description": "Preheat oven to 375\u00b0F (190\u00b0C)"
    },
    {
      "timestamp": "12:15 pm",
      "title": "Make pie crust",
      "description": "Make the pie crust"
    },
    {
      "timestamp": "12:30 pm",
      "title": "Prepare apple filling",
      "description": "Prepare the apple filling"
    },
    {
      "timestamp": "12:45 pm",
      "title": "Assemble pie",
      "description": "Assemble the pie"
    },
    {
      "timestamp": "1:00 pm",
      "title": "Bake pie",
      "description": "Bake the pie for 45-50 minutes"
    }
  ]
}


In [64]:
user_prompt = "What is the capital of France?"
result = run_conversation(user_prompt)
print(json.dumps(json.loads(result.json()), indent=2))

'NoneType' object has no attribute 'arguments'
{
  "id": "chatcmpl-2532e054-643f-4b7c-8397-bfc298f46c5e",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The capital of France is Paris.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1720190527,
  "model": "llama3-70b-8192",
  "object": "chat.completion",
  "system_fingerprint": "fp_753a4aecf6",
  "usage": {
    "completion_tokens": 8,
    "prompt_tokens": 1066,
    "total_tokens": 1074,
    "completion_time": 0.022857143,
    "prompt_time": 0.292474678,
    "queue_time": null,
    "total_time": 0.315331821
  },
  "x_groq": {
    "id": "req_01j21ngt4sfr3tgc9yfq2jrev6"
  }
}
